In [1]:
!pip install transformers datasets torch sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which i

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-cy")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-cy")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/779k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/782k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from datasets import load_dataset

train_file_path = '/content/drive/MyDrive/dataset/english_to_welsh_test.csv'

dataset = load_dataset('csv', data_files={'train': train_file_path})

split_dataset = dataset['train'].train_test_split(test_size=0.1)

train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print(train_dataset, eval_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['english', 'welsh'],
    num_rows: 11839
}) Dataset({
    features: ['english', 'welsh'],
    num_rows: 1316
})


In [5]:
# Tokenize the dataset using the model tokenizer
def preprocess_function(examples):
    inputs = [ex for ex in examples['english']]  # English text
    targets = [ex for ex in examples['welsh']]  # Welsh text
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [6]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/11839 [00:00<?, ? examples/s]

Map:   0%|          | 0/1316 [00:00<?, ? examples/s]

In [7]:
# Implement Data Collator

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [8]:
# Implement Early stopping

from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,    #Stop training after 3 evaluations with no improvement
    early_stopping_threshold=0.01    # Minimum change to qualify as an improvement
)

In [9]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/dataset/outputs",
    do_eval = True,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 25,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    save_steps = 500,
    save_total_limit=3,
    fp16=True,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=10,
    greater_is_better=False,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train_dataset,
    eval_dataset = tokenized_eval_dataset,
    data_collator = data_collator,
    tokenizer = tokenizer,
    callbacks = [early_stopping_callback]
)

trainer.train(resume_from_checkpoint="/content/drive/MyDrive/dataset/outputs/checkpoint-14800")

There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3087: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `wei

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [18]:
!pip install sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.4 MB/s eta 0:00:00


In [21]:
results = trainer.evaluate()
print(results)


KeyError: 'bleu'

In [31]:
trainer.save_model("/content/drive/MyDrive/dataset/./fine-tuned-opus-mt-cy-en")
tokenizer.save_pretrained("/content/drive/MyDrive/dataset/./fine-tuned-opus-mt-cy-en")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[54394]], 'forced_eos_token_id': 0}


('/content/drive/MyDrive/dataset/./fine-tuned-opus-mt-cy-en/tokenizer_config.json',
 '/content/drive/MyDrive/dataset/./fine-tuned-opus-mt-cy-en/special_tokens_map.json',
 '/content/drive/MyDrive/dataset/./fine-tuned-opus-mt-cy-en/vocab.json',
 '/content/drive/MyDrive/dataset/./fine-tuned-opus-mt-cy-en/source.spm',
 '/content/drive/MyDrive/dataset/./fine-tuned-opus-mt-cy-en/target.spm',
 '/content/drive/MyDrive/dataset/./fine-tuned-opus-mt-cy-en/added_tokens.json')